In [ ]:
import os
import glob
import psycopg2
import pandas as pd



def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    df = df[df.page == "NextSong"]

    df = df.loc[:, ["ts", "userId", "level", "song", "artist", "sessionId"]]
    print(df.head())
    df.to_csv("a.csv", index=False, sep=',')
    with open('a.csv', 'r', errors='ignore') as f:
        next(f)
        cur.copy_from(f, "new_table", sep=',', columns=("start_time", "user_id", "level", "song_id", "artist_id", "session_id"))


def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))


def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=Trial#123")
    cur = conn.cursor()

    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()


if __name__ == "__main__":
    main()